In [1]:
import random
import numpy as np

In [ ]:
class Network(object):
    # sizes表示各层神经元的数量
    # 初始化方法：net = Network([2, 3, 1])
    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.size = sizes
        
        """randn(x,y)表示生成的参数为(x,y)型矩阵，均值为0，方差为1的高斯分布
        由于第一层是输入层，不设置bias，因为bias仅在后面的层用于计算输出"""
        self.biases = [np.random.randn(y,1) for y in size[1:]]
        
        """[:-1]丢弃最后一项，[1:]丢弃第一项
        net.weights[1] 是⼀个存储着连接第⼆层和第三层神经元权重的Numpy矩阵
        矩阵的wyx 是连接第⼆层的xth 神经元和第三层的yth 神经元的权重"""
        self.weights = [np.random.randn(y,x) 
                       for x, y in zip(size[:-1],size[1:])]
        
    def feedforward(self, a):
        for b, w in zip(self.biases, self.weights):
            # np.dot()做矩阵乘法
            a = sigmoid(np.dot(w,a)+b)
            return a
        
    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        # 使用mini-batch SGD训练神经网络
        
        # training_data is a list of tuples
        training_data = list(training_data)
        n = len(tarining_data)
        
        if test_data:
            test_data = list(test_data)
            n_test = len(test_data)
            
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [training_data[k:k+mini_batch_size]
                           for k in range(0, n, mini_batch_size)]
            
            # 更新网络权重
            for mini_batch in mini_batchs:
                self.update_mini_batch(mini_batch, eta)
            
            if test_data:
                print("Epoch {} : {} / {}".format(j, self.evaluate(test_data),n_test))
            else:
                 print("Epoch {} complete".format(j))
    
    def update_mini_batch(self, mini_batch, eta):
        # 更新神经网络权重
        
        # 记录每一层神经网络的nabla_b和nabla_w
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            # 调用backprop()计算目标函数对b和w的偏导数
            delta_nabla_b, delta_nabla_w = self.backprop(x,y)
            # nabla_b和nabla_w刚开始都是空列表
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
            
            self.biases = [b - (eta / len(mini_batch)) *nb
                          for b, nb in zip(self.biases, nabla_b)]
            self.weights = [w - (eta / len(mini_batch)) * nw
                            for w, nw in zip(self.weights, nabla_w)]
    
    def backprop(self, x, y):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = []
        # 使用激活函数
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # bp的[3]和[4]    
        delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # 对每一层求偏导
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        # 返回神经网络所有layer的目标函数对相应b和w的偏导数
        # 要注意的是，每一层只需要一个bias，神经元数量个weights
        return (nabla_b, nabla_w)
     
    def evaluate(self, test_data):
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)
    
    def cost_derivative(self, output_activations, y):
        return (output_activations-y)    
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    return sigmoid(z)*(1-sigmoid(z))        